In [0]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [31]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2020-05-23 08:04:39--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 64.233.189.102, 64.233.189.138, 64.233.189.101, ...
Connecting to drive.google.com (drive.google.com)|64.233.189.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q11opdeipcebk8b557q7sdjlfaocmabt/1590221025000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P [following]
--2020-05-23 08:04:39--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q11opdeipcebk8b557q7sdjlfaocmabt/1590221025000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 74.125.23.132, 2404:6800:4008:c02::84
Connecting to doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)|74.125.

In [0]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('/tmp/sentiment.csv')

sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

# Separate out the sentences and labels into training and test sets
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, 
                               padding=padding_type, truncating=trunc_type)

In [34]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '0') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

good case excellent value 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
Good case Excellent value.


In [35]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 16)           16000     
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 9606      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [43]:
num_epochs = 6
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/6
50/50 [==============================] - 0s 5ms/step - loss: 0.0032 - accuracy: 0.9994 - val_loss: 1.0006 - val_accuracy: 0.7494
Epoch 2/6
50/50 [==============================] - 0s 5ms/step - loss: 0.0032 - accuracy: 0.9994 - val_loss: 0.9697 - val_accuracy: 0.7494
Epoch 3/6
50/50 [==============================] - 0s 6ms/step - loss: 0.0031 - accuracy: 0.9994 - val_loss: 1.0033 - val_accuracy: 0.7444
Epoch 4/6
50/50 [==============================] - 0s 5ms/step - loss: 0.0028 - accuracy: 0.9994 - val_loss: 1.0469 - val_accuracy: 0.7469
Epoch 5/6
50/50 [==============================] - 0s 6ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 1.0117 - val_accuracy: 0.7444
Epoch 6/6
50/50 [==============================] - 0s 6ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 1.0535 - val_accuracy: 0.7469


In [0]:
# import matplotlib.pyplot as plt


# def plot_graphs(history, string):
#   plt.plot(history.history[string])
#   plt.plot(history.history['val_'+string])
#   plt.xlabel("Epochs")
#   plt.ylabel(string)
#   plt.legend([string, 'val_'+string])
#   plt.show()
  
# plot_graphs(history, "accuracy")
# plot_graphs(history, "loss")

In [55]:
# Use the model to predict a review   
some_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

print(some_reviews) 

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(some_reviews)
sample_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

print('\nLets try implementing the model\n')              

classes = model.predict(sample_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(some_reviews)):
  print(fake_reviews[x])
  i = np.round(classes[x],15)
  print(i)
  if(i<0.2):
    print('It is a very bad responce')
  elif(i<0.4):
    print('It is not  a bad responce')
  elif(i<0.6):
    print('It is an everage responce')
  elif(i<0.8):
    print('It is a good responce')
  else:
    print('It is an excellent responce')
  print('\n')

# Try adding reviews of your own
# Add some negative words (such as "not") to the good reviews and see what happens
# For example:
# they gave us free chocolate cake and did not charge us

['I love this phone', 'I hate spaghetti', 'Everything was cold', 'Everything was hot exactly as I wanted', 'Everything was green', 'the host seated us immediately', 'they gave us free chocolate cake', 'not sure about the wilted flowers on the table', 'only works when I stand on tippy toes', 'does not work when I stand on my head']

Lets try implementing the model

I love this phone
[0.99999404]
It is an excellent responce


I hate spaghetti
[3.0542866e-05]
It is a very bad responce


Everything was cold
[0.00084935]
It is a very bad responce


Everything was hot exactly as I wanted
[0.02002569]
It is a very bad responce


Everything was green
[0.01272446]
It is a very bad responce


the host seated us immediately
[0.9999609]
It is an excellent responce


they gave us free chocolate cake
[0.9999843]
It is an excellent responce


not sure about the wilted flowers on the table
[6.750078e-05]
It is a very bad responce


only works when I stand on tippy toes
[0.9999628]
It is an excellent r